In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
import os
from shutil import copy, rmtree, move
import random
import cv2

In [2]:
def display(image):
    fig=plt.figure(figsize=(4,3))
    ax=fig.add_subplot(111)
    ax.imshow(image)

In [3]:
def img_bgr(image):
    (B,G,R) = cv2.split(image)
    return B,G,R

In [4]:
def img_rotate(img,ro=15,savepath="Rotated.jpg"):
    rotated = imutils.rotate_bound(img, ro)
    cv2.imwrite(savepath,rotated)

In [5]:
def img_rgb(image):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    return image

In [6]:
def img_svd(img,k=90):
    u,s,v = np.linalg.svd(img,full_matrices=0)
    img = np.dot(u[:,:k]*s[:k],v[:k,:]).astype(np.uint8)
    return img

In [7]:
def img_merged(B,G,R):
    merged = cv2.merge([B,G,R])
    return merged

In [8]:
def mk_file(file_path: str):
    if os.path.exists(file_path):
        # 如果文件夹存在，则先删除原文件夹在重新创建
        rmtree(file_path)
    os.makedirs(file_path)

In [9]:
# 指向源图的文件夹
cwd = os.getcwd()
data_root = os.path.abspath(os.path.join(cwd))
origin_data_path = os.path.join(data_root, "test525")
assert os.path.exists(origin_data_path), "path '{}' does not exist.".format(origin_data_path)

In [11]:
data_class = [cla for cla in os.listdir(origin_data_path)
                if os.path.isdir(os.path.join(origin_data_path, cla))]

In [12]:
# 建立保存的文件夹
train_root = os.path.join(data_root,"r90_test525")
mk_file(train_root)
for cla in data_class:
    # 建立每个类别对应的文件夹
    mk_file(os.path.join(train_root, cla))

In [13]:
#SVD
total_num = 0
for cla in data_class:
    cla_path = os.path.join(origin_data_path, cla)
    images = os.listdir(cla_path)
    num = len(images)
    total_num += num
    for index, image in enumerate(images):
        image_path = os.path.join(cla_path, image)
        img_name = image_path.split('/')[-1].split(".")[0]
        savepath = os.path.join(train_root, cla,img_name + ".jpg")

        img = cv2.imread(image_path)
        B,G,R = img_bgr(img)
        b = img_svd(B)
        g = img_svd(G)
        r = img_svd(R)
        img = img_merged(b,g,r)
        #png
        # cv2.imwrite(savepath,img, [int(cv2.IMWRITE_PNG_COMPRESSION), 1])
        #jpg
        cv2.imwrite(savepath,img,[int(cv2.IMWRITE_JPEG_QUALITY),100])
 
 
        print("\r[{}] processing [{}/{}]".format(cla, index+1, num), end="")  # processing bar
    # break
    print()

print(f"processing {total_num} done!")

[vasc] processing [10/10]
[nv] processing [663/663]
[bkl] processing [66/66]
[akiec] processing [23/23]
[mel] processing [34/34]
[df] processing [6/6]
[bcc] processing [26/26]
processing 828 done!


In [6]:
# # Image segmentation
# ham = os.listdir("HAM10000/")
# ham_dir = "./HAM10000"

In [ ]:
# for cla in data_class:
#     cla_path = os.path.join(origin_data_path, cla)
#     images = os.listdir(cla_path)
#     num = len(images)

#     for index, image in enumerate(images):
#         image_path = os.path.join(cla_path, image)
#         img_name = image_path.split('/')[-1].split(".")[0]
#         seg_name = img_name + "_segmentation.png"
#         seg_file = os.path.join(ham_dir,seg_name)
#         img1 = Image.open(seg_file)
#         img1 = img1.convert('RGB')
#         img2 = Image.open(image_path)
#         img2 = img2.convert('RGB')
#         img = Image.blend(img1, img2, 1.36)
#         new_path = os.path.join(train_root, cla,image_path.split('/')[-1])
#         img.save(new_path)
        
#         print("\r[{}] processing [{}/{}]".format(cla, index+1, num), end="")  # processing bar
#     print()

# print("processing done!")